In [1]:
import pandas as pd
file_path = '/Users/moji/PyTSF-MfG/data/MWTP_Elec_Daily.csv'
df = pd.read_csv(file_path)
df

,date,kwh1,kw1,kwh2,kw2,billed_kwh,solar_kwh,total_kwh,mg_finish
0,01/01/2010,13014.0,758.4,0.0,0.0,13014.0,0.000,13014.0,11.670000
1,01/02/2010,14058.0,811.2,0.0,0.0,14058.0,0.000,14058.0,11.740000
2,01/03/2010,14560.8,842.4,0.0,0.0,14560.8,0.000,14560.8,13.150000
3,01/04/2010,14814.0,756.0,0.0,0.0,14814.0,0.000,14814.0,12.950000
4,01/05/2010,15052.8,777.6,0.0,0.0,15052.8,0.000,15052.8,13.070000
...,...,...,...,...,...,...,...,...,...
5352,08/27/2024,16311.6,792.0,0.0,0.0,16311.6,758.367,17070.0,18.859375
5353,08/28/2024,16780.8,763.2,0.0,0.0,16780.8,695.311,17476.1,19.078125
5354,08/29/2024,15290.4,768.0,0.0,0.0,15290.4,644.772,15935.2,17.328125
5355,08/30/2024,16218.0,756.0,0.0,0.0,16218.0,637.378,16855.4,18.562500


In [2]:
# Display basic information about the dataset
print("Dataset info:")
print(df.info())

Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5357 entries, 0 to 5356
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        5357 non-null   object 
 1   kwh1        5357 non-null   float64
 2   kw1         5356 non-null   float64
 3   kwh2        5357 non-null   float64
 4   kw2         5357 non-null   float64
 5   billed_kwh  5356 non-null   float64
 6   solar_kwh   5357 non-null   float64
 7   total_kwh   5357 non-null   float64
 8   mg_finish   5357 non-null   float64
dtypes: float64(8), object(1)
memory usage: 376.8+ KB
None


In [4]:
print("\nDataset head:")
print(df.head())
# Check for duplicate timestamps
duplicates = df[df.duplicated(subset=['date'], keep=False)]
print(f"\nNumber of rows with duplicate timestamps: {len(duplicates)}")



Dataset head:
         date     kwh1    kw1  kwh2  kw2  billed_kwh  solar_kwh  total_kwh  \
0  01/01/2010  13014.0  758.4   0.0  0.0     13014.0        0.0    13014.0   
1  01/02/2010  14058.0  811.2   0.0  0.0     14058.0        0.0    14058.0   
2  01/03/2010  14560.8  842.4   0.0  0.0     14560.8        0.0    14560.8   
3  01/04/2010  14814.0  756.0   0.0  0.0     14814.0        0.0    14814.0   
4  01/05/2010  15052.8  777.6   0.0  0.0     15052.8        0.0    15052.8   

   mg_finish  
0      11.67  
1      11.74  
2      13.15  
3      12.95  
4      13.07  

Number of rows with duplicate timestamps: 0


In [5]:
if len(duplicates) > 0:
    print("\nSample of duplicate timestamps:")
    print(duplicates.head(10))  # Show first 10 duplicates

    # Option 1: Keep the first occurrence of each timestamp
    df_deduped = df.drop_duplicates(subset=['date'], keep='first')

    # Option 2: Aggregate data for duplicate timestamps (e.g., taking the mean)
    # df_deduped = df.groupby(['unique_id', 'ds'])['y'].mean().reset_index()

    print(f"\nShape of dataset after deduplication: {df_deduped.shape}")

    # Check if deduplication resolved the issue
    if df_deduped['Date_Time'].is_unique:
        print("Deduplication successful. Timestamps are now unique.")
    else:
        print("Deduplication did not resolve all duplicates. Further investigation needed.")

    # # Save the deduplicated dataset
    # output_path = '/Users/moji/PyTSF-MfG/data/deduplicated_dataset.csv'  # Replace with your desired output path
    # df_deduped.to_csv(output_path, index_label='id')
    # print(f"\nDeduplicated dataset saved as: {output_path}")

else:
    print("No duplicate timestamps found. The issue may be elsewhere.")

No duplicate timestamps found. The issue may be elsewhere.


In [ ]:
import pandas as pd
# Make an explicit copy to ensure we're working with a new DataFrame
df_deduped = df_deduped.copy()

print("\nDate range:")
print(f"Start: {df_deduped['date'].min()}")
print(f"End: {df_deduped['date'].max()}")

# Convert 'Date_Time' to datetime using .loc
df_deduped.loc[:, 'date'] = pd.to_datetime(df_deduped['date'], format='%m-%d-%Y')

# Check for any inconsistencies in the time intervals
time_diff = df_deduped['date'].diff()
inconsistent_intervals = time_diff[time_diff != pd.Timedelta(minutes=15)]
if not inconsistent_intervals.empty:
    print("\nInconsistent time intervals found:")
    print(inconsistent_intervals.head())
else:
    print("\nAll time intervals are consistent (15 minutes).")

In [9]:
# Load the dataset
df = df.copy()
# Convert 'Date' column to datetime
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')
# Set 'Date' as the index
df.set_index('date', inplace=True)
# Sort the index to ensure chronological order
df.sort_index(inplace=True)
# Create a complete date range including weekends
date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
# Reindex the dataframe with the complete date range
df_reindexed = df.reindex(date_range)
# Check for missing values after reindexing
print("Missing values after reindexing (including weekends):")
print(df_reindexed.isnull().sum())
# Interpolate missing values
df_interpolated = df_reindexed.interpolate(method='time')
# Check for missing values after interpolation
print("\nMissing values after interpolation:")
print(df_interpolated.isnull().sum())
# Fill any remaining NaN values at the beginning or end with the nearest valid value
df_interpolated = df_interpolated.fillna(method='bfill').fillna(method='ffill')

# Save the new dataset without missing values
output_path = '/Users/moji/PyTSF-MfG/data/new_dataset.csv'
df_interpolated.to_csv(output_path)
print(f"\nNew dataset saved as '{output_path}'")

# Display the first few rows of the interpolated dataset
print("\nFirst few rows of the interpolated dataset:")
print(df_interpolated.head())

# Display basic statistics of the interpolated dataset
print("\nBasic statistics of the interpolated dataset:")
print(df_interpolated.describe())

Missing values after reindexing (including weekends):
kwh1          0
kw1           1
kwh2          0
kw2           0
billed_kwh    1
solar_kwh     0
total_kwh     0
mg_finish     0
dtype: int64

Missing values after interpolation:
kwh1          0
kw1           0
kwh2          0
kw2           0
billed_kwh    0
solar_kwh     0
total_kwh     0
mg_finish     0
dtype: int64

New dataset saved as '/Users/moji/PyTSF-MfG/data/new_dataset.csv'

First few rows of the interpolated dataset:
               kwh1    kw1  kwh2  kw2  billed_kwh  solar_kwh  total_kwh  \
2010-01-01  13014.0  758.4   0.0  0.0     13014.0        0.0    13014.0   
2010-01-02  14058.0  811.2   0.0  0.0     14058.0        0.0    14058.0   
2010-01-03  14560.8  842.4   0.0  0.0     14560.8        0.0    14560.8   
2010-01-04  14814.0  756.0   0.0  0.0     14814.0        0.0    14814.0   
2010-01-05  15052.8  777.6   0.0  0.0     15052.8        0.0    15052.8   

            mg_finish  
2010-01-01      11.67  
2010-01-02      

/var/folders/0s/whfbkk11083fmvg1wl2nrqqm0000gn/T/ipykernel_98377/3610453067.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_interpolated = df_interpolated.fillna(method='bfill').fillna(method='ffill')


In [ ]:
data_path = '/Users/moji/PyTSF-MfG/data'
datasets = load_datasets_statforecast_uni(data_path)

In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('/Users/moji/PyTSF-MfG/data/BrentOilPrices.csv')
# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'], format='mixed')
# Set 'Date' as the index
df.set_index('Date', inplace=True)
# Sort the index to ensure chronological order
df.sort_index(inplace=True)
# Create a complete date range including weekends
date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
# Reindex the dataframe with the complete date range
df_reindexed = df.reindex(date_range)
# Check for missing values after reindexing
print("Missing values after reindexing (including weekends):")
print(df_reindexed.isnull().sum())
# Interpolate missing values
df_interpolated = df_reindexed.interpolate(method='time')
# Check for missing values after interpolation
print("\nMissing values after interpolation:")
print(df_interpolated.isnull().sum())
# Fill any remaining NaN values at the beginning or end with the nearest valid value
df_interpolated = df_interpolated.fillna(method='bfill').fillna(method='ffill')

# Save the new dataset without missing values
output_path = '/Users/moji/PyTSF-MfG/data/BrentOilPrices_Interpolated.csv'
df_interpolated.to_csv(output_path)
print(f"\nNew dataset saved as '{output_path}'")

# Display the first few rows of the interpolated dataset
print("\nFirst few rows of the interpolated dataset:")
print(df_interpolated.head())

# Display basic statistics of the interpolated dataset
print("\nBasic statistics of the interpolated dataset:")
print(df_interpolated.describe())

In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('/Users/moji/PyTSF-MfG/data/ISO-NY_Central.csv')
print(df.shape)
# Convert 'Time_Stamp' column to datetime
df['Time_Stamp'] = pd.to_datetime(df['Time_Stamp'])

# Set 'Time_Stamp' as the index
df.set_index('Time_Stamp', inplace=True)

# Sort the index to ensure chronological order
df.sort_index(inplace=True)

# Create a complete time range with 15-minute intervals
time_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='15min')

# Reindex the dataframe with the complete time range
df_reindexed = df.reindex(time_range)
print(df_reindexed.shape)
# Check for missing values after reindexing
print("Missing values after reindexing:")
print(df_reindexed.isnull().sum())
# Interpolate missing values
df_interpolated = df_reindexed.interpolate(method='time')
# Check for missing values after interpolation
print("\nMissing values after interpolation:")
print(df_interpolated.isnull().sum())
# Fill any remaining NaN values at the beginning or end with the nearest valid value
df_interpolated = df_interpolated.fillna(method='bfill').fillna(method='ffill')
# Final check for missing values
print("\nFinal check for missing values:")
print(df_interpolated.isnull().sum())

# Save the new dataset without missing values
output_path = '/Users/moji/PyTSF-MfG/data/LoadData_Interpolated.csv'
df_interpolated.to_csv(output_path)

print(f"\nNew dataset saved as '{output_path}'")

# Display the first few rows of the interpolated dataset
print("\nFirst few rows of the interpolated dataset:")
print(df_interpolated.head())

# Display basic statistics of the interpolated dataset
print("\nBasic statistics of the interpolated dataset:")
print(df_interpolated.describe())
# Check for any gaps in the time series
time_diff = df_interpolated.index.to_series().diff()
gaps = time_diff[time_diff > pd.Timedelta(minutes=15)]
if not gaps.empty:
    print("\nGaps found in the time series:")
    print(gaps)
else:
    print("\nNo gaps found in the time series.")